<a href="https://colab.research.google.com/github/ChitandaMayaka/CMPE255-Team-Project/blob/main/DataPreprocessingAndVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing and Visualization

- Data Preprocessing
    - Drop all unnecessary column ['id', 'url', 'region_url', 'vin','size', 'type', 'paint_color', 'image_url', 'description', 'county', 'lat', 'long']
    - Drop duplicated row
    - Preprocessing price. Drop the value under 1000 and over 100000
    - Preprocessing year. Drop the value under 1980 and over 2020

- Data Visualization
  - Missing 
  - Distribution(count) by manufacturer
  - Price range by manufacturer
  - Distribution(count) by year
  - Price ranage by year
  - Proportion by Fuel Type
  - Visualize car location in map

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## Data Preprocessing

### Handle with common unnecessary columns and rows

In [ ]:
data = pd.read_csv('drive/Shared drives/CMPE255 Project/dataset/vehicles.csv')

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
# Drop unnecessary columns
data.drop(columns=['id', 'url', 'region_url', 'vin','size', 'type', 'paint_color', 'image_url', 'description', 'county'], inplace=True)

In [ ]:
# Remove duplicated row
data.drop_duplicates(inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data

In [ ]:
# remove rows which do not have year, manufacturer and odometer all three of them
index_num = data[data['year'].isnull() & data['manufacturer'].isnull() & 
                 data['odometer'].isnull()].index

In [ ]:
data.drop(index_num, inplace=True)

In [ ]:
data

### Preprocessing price

In [ ]:
# filter abnormal price data
# We can think the car whose price under $1000 are scarp
# and over $100000 are overprice
data['price'].describe()

In [ ]:
data.drop(data[data['price'] < 1000].index, inplace=True)
data.drop(data[data['price'] > 3e5].index, inplace=True)
data['price'].describe()

In [ ]:
#average_price_bymanufacturer = data.groupby('manufacturer')['price'].mean()

In [ ]:
#average_price_bymanufacturer.sort_values(ascending=False, inplace=True)

In [ ]:
# Visualize data after all preprocessing
#plt.figure(figsize=(15,10))
#ax = average_price_bymanufacturer.plot.bar()
#plt.title('Average Price by Manufacturer')
#plt.show()

In [ ]:
#average_price_byyear = data.groupby('year')['price'].mean()

In [ ]:
#plt.figure(figsize=(15,10))
#ax =average_price_byyear.plot.bar()
#plt.title('Average Price by Year')
#plt.show()

In [ ]:
data[data['state'] == 'ca']

In [ ]:
data[data['price'] == 3e5]

### Preprocessing year

In [ ]:
data['year'].isnull().sum()

In [ ]:
data['year'].describe()

In [ ]:
# discard value in ['year'] smaller than 1980 and bigger than 2021
data['year'] = data['year'].loc[data['year'] > 1980]
data['year'] = data['year'].loc[data['year'] < 2021]
data['year'].describe()

In [ ]:
data['year'].value_counts()

### Preprocessing region and state

In [ ]:
data['region'].isnull().sum()

In [ ]:
data['state'].isnull().sum()

In [ ]:
data['region'].value_counts()

In [ ]:
data['state'].value_counts()

In [ ]:
len(data['state'].unique())

In [ ]:
# Consider combine this two column to create new column location
data['location'] = data['region'] + data['state']

In [ ]:
data['location'].value_counts()

### Preprocessing title status

In [ ]:
# title status missing percentage
data.title_status.isnull().sum()/len(data)

In [ ]:
data.title_status.value_counts()

In [ ]:
# Only 0.5% title status is still missing, as most cars are clean, use clean to fill the missing values
data.title_status.fillna("clean", inplace=True)

## Visualization

### Missing value for preprocessed data

In [ ]:
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values(by = 'percent_missing', ascending=False, inplace=True)
missing_value_df.plot(kind="bar")

### Distribution by manufacturer

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(rotation=45)
sns.countplot(data['manufacturer'], order = data['manufacturer'].value_counts().index)

The top 2 car manufacturers are Ford and Chevrolet, both are American automobile company. The next 3 popular are Japanese cars: Toyota, Honda and Nissan.

### Price Range by manufacturer

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(rotation=45)
sns.boxplot(x='manufacturer', y='price', data=data, showfliers=False)

The prices for luxury car Manufacters like Ferrari or Aston Martin are significantly higher.

### Distribution by year

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(rotation=45)
sns.countplot(data['year'])

Most cars are in the recent 10 years, from 2010~2020.

### Price Range by year

In [ ]:
plt.figure(figsize=(20,10))
plt.xticks(rotation=45)
sns.boxplot(x='year', y='price', data=data, showfliers=False)

Relatively, the newer the car, the higher the price.

### Proportion by fuel type

In [ ]:
data.fuel.value_counts().plot.pie(subplots=True, figsize=(11, 6), autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.3)
plt.title('Number of cars by fuel type')
plt.legend()
plt.show()

Most(88%) of the cars are gasoline cars.

### Visualize car location in map

In [ ]:
data.columns

In [ ]:
token = "pk.eyJ1Ijoic3pvbmciLCJhIjoiY2tocW5mdnFrMG95NTJ5bngxdDQ5dXdoZyJ9.xDVaETZma1QC-4eCOR62PQ"
px.set_mapbox_access_token(token)

In [ ]:
fig = px.scatter_mapbox(data,
                        lat="lat",
                        lon="long",
                        zoom=3,
                        hover_name="manufacturer",
                        hover_data=['price', 'year','odometer', 'title_status'],
                        )
# fig.update_layout(mapbox_style="open-street-map")
fig.show()

Most on sale used cars are in the east, there are some outliers with inaccurate latitude longitude data that results in the sea.

In [ ]:
# check outliers on map(lat lon in the sea)
data[data.lat==30.8635]